程序功能：测算储能调节前后的考核电量和考核金额（元）。

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('std_data_5min.csv'))
PReal = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_realpower"]
Pust = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_ust_predpower"]
# 构造10min考核点
# 由5min构造10min考核点时，由于是后向平均，需要用i点和i点前一个数据求和除以二
# 105119
PReal = np.array(PReal)  # 确保PReal是一个numpy数组
averagesPReal = (PReal[:-1] + PReal[1:]) / 2.0
averagesPReal = np.append(averagesPReal, averagesPReal[-1]) # 保持数组长度一致
# 计算相邻10分钟的功率波动
fluctuations = []
for i in range(0, len(averagesPReal) - 2):  # 减2是因为我们是计算相隔10分钟（即每隔一个索引）的差值
    fluctuation = abs(averagesPReal[i] - averagesPReal[i+2])
    fluctuations.append(fluctuation)
df["realpower_Fluctuation"] = pd.DataFrame(fluctuations)
# 免考核计算
Pc_thy = pd.DataFrame(pd.read_csv('std_Pc_thy.csv'))["Pc_thy"]
ad_real = averagesPReal + Pc_thy
# 考核电量
Wcap = 2254.4*1000 # 风电场容量、kw作为基值的标幺单位
Ccap = 29983520.0*0.4*0.3 # 百分之10的配储（kwh）
Pcmax = 0.407 # 1c的储能（kw）
Soc0 = 0.5
Socmax = 0.97
Socmin = 0.03
eta = 0.932
p = 0.453 
deltaT = 5/60 # 单个决策点放电时间为5min尺度
points = int(24*60/5) # 5min优化点的个数，该数值会随着时间尺度的缩小而增大，例如1min时间尺度的数据是1440个点
threshold = 50*1000/Wcap # 因为使用了非线性求解器，需要多留一点裕度
A = 1
N = 1
step = Wcap*0.1 # 储能免考核求解步长
sum_exceed_threshold = sum(value for value in fluctuations if value > threshold)
penalty = A*sum_exceed_threshold*10/60*p*Wcap
print("考核电量为{}kwh，考核价格为{}元".format(sum_exceed_threshold*10/60*Wcap,penalty))
# 计算相邻10分钟的功率波动
ad_fluctuations = []
for i in range(0, len(ad_real) - 2):  # 减2是因为我们是计算相隔10分钟（即每隔一个索引）的差值
    ad_fluctuation = abs(ad_real[i] - ad_real[i+2])
    ad_fluctuations.append(ad_fluctuation)
sum_exceed_threshold = sum(value for value in ad_fluctuations if value > threshold)
penalty = A*sum_exceed_threshold*10/60*p*Wcap
print("储能调节后，考核电量为{}kwh，考核价格为{}元".format(sum_exceed_threshold*10/60*Wcap,penalty))

考核电量为250708350.55611908kwh，考核价格为113570882.80192195元
储能调节后，考核电量为0.0kwh，考核价格为0.0元
